In [1]:
import pandas as pd
import numpy as np
import json
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
import PyPDF2
import os

In [2]:
load_dotenv()
open_api_key=os.environ.get('OPENAI_API_KEY')

In [3]:
llm=ChatOpenAI(openai_api_key=open_api_key,model='gpt-3.5-turbo',temperature=0.7)

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
with open(r'C:\Users\HP\OneDrive\Desktop\desktop\generativeAI\openai_mcq_gen\Response.json') as f:
    Response_JSON=json.load(f)

In [5]:
print(Response_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [6]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [7]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [8]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [10]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [11]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [12]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],output_variables=["quiz", "review"], verbose=True,)

In [13]:
with open(r'C:\Users\HP\OneDrive\Desktop\desktop\generativeAI\openai_mcq_gen\data.txt') as file:
    TEXT=file.read()

In [14]:
print(TEXT)

Photosynthesis, the plantâ€™s magical cooking, unfolds in tiny kitchens called chloroplasts. These mini-factories, wrapped in a double layer, hold a green chef called chlorophyll. Inside, there are flat sacs called thylakoids, forming stacks named grana in a clear space called stroma.Chlorophyll, the chefâ€™s assistant, loves catching red and blue light, giving plants their green color. This kitchen duo transforms air and water into yummy carbohydrates, and as a bonus, they serve up oxygen. Itâ€™s like a dance â€“ sunlight, colors, and plant kitchens working together to make lifeâ€™s essential recipe, the enchanting process of photosynthesis.


In [15]:
Text=TEXT
NUMBER=5
SUBJECT='Biology'
TONE='Simple'
RESPONSE_JSON=Response_JSON

In [16]:
Response_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [17]:
with  get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {

        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)

        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Photosynthesis, the plantâ€™s magical cooking, unfolds in tiny kitchens called chloroplasts. These mini-factories, wrapped in a double layer, hold a green chef called chlorophyll. Inside, there are flat sacs called thylakoids, forming stacks named grana in a clear space called stroma.Chlorophyll, the chefâ€™s assistant, loves catching red and blue light, giving plants their green color. This kitchen duo transforms air and water into yummy carbohydrates, and as a bonus, they serve up oxygen. Itâ€™s like a dance â€“ sunlight, colors, and plant kitchens working together to make lifeâ€™s essential recipe, the enchanting process of photosynthesis.
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 5 multiple choice questions for Biology students in Simple tone. 
Make sure the questions are not repeated and check all the questions to be conforming th

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{
    "1": {
        "no": "1",
        "mcq": "What is the name of the tiny kitchens where photosynthesis takes place in plants?",
        "options": {
            "a": "Mitochondria",
            "b": "Chloroplasts",
            "c": "Nucleus",
            "d": "Ribosomes"
        },
        "correct": "b"
    },
    "2": {
        "no": "2",
        "mcq": "What is the name of the green chef inside the chloroplasts that catches red and blue light?",


In [18]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1429
Prompt Tokens:954
Completion Tokens:475
Total Cost:0.0023810000000000003


In [19]:
response

{'text': 'Photosynthesis, the plantâ€™s magical cooking, unfolds in tiny kitchens called chloroplasts. These mini-factories, wrapped in a double layer, hold a green chef called chlorophyll. Inside, there are flat sacs called thylakoids, forming stacks named grana in a clear space called stroma.Chlorophyll, the chefâ€™s assistant, loves catching red and blue light, giving plants their green color. This kitchen duo transforms air and water into yummy carbohydrates, and as a bonus, they serve up oxygen. Itâ€™s like a dance â€“ sunlight, colors, and plant kitchens working together to make lifeâ€™s essential recipe, the enchanting process of photosynthesis.',
 'number': 5,
 'subject': 'Biology',
 'tone': 'Simple',
 'RESPONSE_JSON': '{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", 

In [20]:
quiz=response.get("quiz")
quiz=json.loads(quiz)
print(quiz)

{'1': {'no': '1', 'mcq': 'What is the name of the tiny kitchens where photosynthesis takes place in plants?', 'options': {'a': 'Mitochondria', 'b': 'Chloroplasts', 'c': 'Nucleus', 'd': 'Ribosomes'}, 'correct': 'b'}, '2': {'no': '2', 'mcq': 'What is the name of the green chef inside the chloroplasts that catches red and blue light?', 'options': {'a': 'DNA', 'b': 'Chlorophyll', 'c': 'RNA', 'd': 'Protein'}, 'correct': 'b'}, '3': {'no': '3', 'mcq': 'What are the flat sacs inside chloroplasts called?', 'options': {'a': 'Golgi Apparatus', 'b': 'Endoplasmic Reticulum', 'c': 'Thylakoids', 'd': 'Lysosomes'}, 'correct': 'c'}, '4': {'no': '4', 'mcq': 'Which part of the chloroplasts transforms air and water into carbohydrates?', 'options': {'a': 'Thylakoids', 'b': 'Stroma', 'c': 'Grana', 'd': 'Chlorophyll'}, 'correct': 'b'}, '5': {'no': '5', 'mcq': 'What is the main product of photosynthesis that is released into the air?', 'options': {'a': 'Carbon Dioxide', 'b': 'Water', 'c': 'Oxygen', 'd': 'Gluc

In [21]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value['mcq']
    options='|'.join(
        [
            f'{option}:{option_value}'
            for option,option_value in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_table_data.append({'MCQ':mcq,'Choices':options,'Correct':correct})

In [22]:
quiz_table_data

[{'MCQ': 'What is the name of the tiny kitchens where photosynthesis takes place in plants?',
  'Choices': 'a:Mitochondria|b:Chloroplasts|c:Nucleus|d:Ribosomes',
  'Correct': 'b'},
 {'MCQ': 'What is the name of the green chef inside the chloroplasts that catches red and blue light?',
  'Choices': 'a:DNA|b:Chlorophyll|c:RNA|d:Protein',
  'Correct': 'b'},
 {'MCQ': 'What are the flat sacs inside chloroplasts called?',
  'Choices': 'a:Golgi Apparatus|b:Endoplasmic Reticulum|c:Thylakoids|d:Lysosomes',
  'Correct': 'c'},
 {'MCQ': 'Which part of the chloroplasts transforms air and water into carbohydrates?',
  'Choices': 'a:Thylakoids|b:Stroma|c:Grana|d:Chlorophyll',
  'Correct': 'b'},
 {'MCQ': 'What is the main product of photosynthesis that is released into the air?',
  'Choices': 'a:Carbon Dioxide|b:Water|c:Oxygen|d:Glucose',
  'Correct': 'c'}]

In [23]:
df=pd.DataFrame(quiz_table_data)

In [24]:
df.to_csv('mcq_bits.csv',index=False)